In [1]:
#instantiate cursor and load libraries
import json
import pandas as pd
import requests
import mysql.connector
import config
import time


In [2]:
#specify database for connection
db = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = "Movies")

cursor = db.cursor()

In [3]:
#create database
from mysql.connector import errorcode
db_name = 'Movies'

def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [4]:
import requests
from bs4 import BeautifulSoup as BS

In [5]:
#first page

page1 = requests.get('https://www.imdb.com/search/title/?release_date=2019-01-01,2019-10-31&sort=boxoffice_gross_us,desc&start=251&ref_=adv_nxt')
soup1 = BS(page1.content, 'html.parser')
soup1.prettify()
info1 = soup1.find_all(id = 'main')
movie1 = info1[0].find_all(class_='lister-item mode-advanced')

In [6]:
# insert_movies(make_df(movie1))

In [7]:
def get_score(movie):
        metascores = []
        for score in movie:
            metascore1 = score.find(class_="metascore")
            if metascore1 == None:
                metascore = 'Null'
                metascores.append(metascore)
            else:
                metascores.append(int(metascore1.get_text(strip=True)))
        return metascores
# len(get_score(movie))

In [8]:
def find_fk(movie):
    ids = []
    for m in movie:
        id = m.find (class_="starBarWidget")['id']
        ids.append(id)
    return ids

# find_fk(movie1)

In [9]:
def get_titles(movie):

    titles = []
    for one in movie:
        title = one.find('img')['alt']
        titles.append(title) 
    return titles

# len(get_titles(movie))

In [10]:
def get_stars(movie):
    stars = []
    for one in movie:
        try:
            stars.append(float(one.find(class_='inline-block ratings-imdb-rating')['data-value']))
        except:
            stars.append('Null')
#         stars.append(float(x['data-value']))
    return stars

# len(get_stars(movie))

In [11]:
def get_year(movie):
    
    years = []
    for one in movie:
        try:
            year = one.find(class_="lister-item-year text-muted unbold").get_text()
            parsed_year = year[-5:-1]
            years.append(int(parsed_year))
        except:
            years.append(2019)
    return years
# get_year(movie1)

In [12]:
def get_genre(movie):
    genres = []
    for m in movie:
        genre = m.find(class_='genre')
        try:
            genre1 = str.split(genre.get_text(strip=True), sep = ',')[0]
        except:
            genre1 = genre
        genres.append(genre1)
    return genres
# len(get_genre(movie1))

In [13]:
def get_gross(movie):
    gross_list = []
    
    for m in movie:
        gross = m.find(class_='sort-num_votes-visible')
        parsed_gross = gross.get_text(strip=True)
        parsed_gross2 = str.split(parsed_gross, sep='|')
       # print(parsed_gross2[1])
        if 'Gross' in parsed_gross2[0]:
            parsed_gross2 = parsed_gross2[0]
#             print(parsed_gross2[0])
            parsed_gross3 = str.split(parsed_gross2, sep=':')
#             print(parsed_gross3)
            parsed_gross4 = str.split(parsed_gross3[1], sep='.')[0]
            final_gross = str.split(parsed_gross4, sep='$')[1]
            gross_list.append(int(final_gross))
#             print(parsed_gross2)
        else:
            try:       
                parsed_gross3 = str.split(parsed_gross2[1], sep=':')
                parsed_gross4 = str.split(parsed_gross3[1], sep='.')[0]
                final_gross = str.split(parsed_gross4, sep='$')[1]
                gross_list.append(int(final_gross))
            except: 
                gross_list.append(0)
    
    return gross_list

# len(get_gross(movie))

In [14]:
gross = movie1[0].find(class_='sort-num_votes-visible').get_text(strip=True)
gross

'Votes:22|Gross:$0.01M'

In [41]:
#create table
DB_NAME = 'Movies'

create_query = """
      CREATE TABLE imdb_movies
    (
      imdb_id varchar(24),
      title varchar(50),
      year INT(4),
      star_rating FLOAT(3),
      metascore INT(5),
      gross INT(5),
      genre varchar(30),
      CONSTRAINT primary_key PRIMARY KEY(imdb_id)
      )
    """



In [ ]:
#make into DF
def make_df(movie):
    all_movies_dataframe = pd.DataFrame({
        "imdb_id": find_fk(movie),
        "title": get_titles(movie),
        "year": get_year(movie),
        "star_rating": get_stars(movie),
        "metascore": get_score(movie),
        "gross": get_gross(movie),
        "genre": get_genre(movie)
    })
    return all_movies_dataframe


In [ ]:
#uses DF to insert into mySQL
def insert_movies(dataframe):
    all_movies_tuples = list(dataframe.itertuples(index=False, name=None))
    insert_statement = """INSERT INTO imdb_movies (imdb_id, title, year, star_rating, metascore, gross, genre) 
            VALUES (%s, %s, %s, %s, %s, %s, %s);"""
    for movie in all_movies_tuples:
        
        cursor.execute(insert_statement, movie)
        print(movie)
    
#     cursor.executemany(insert_statement, all_movies_tuples)
    db.commit()


In [ ]:
var = 251
while var < 750: 
    page = requests.get(f"https://www.imdb.com/search/title/?release_date=2019-01-01,2019-10-31&sort=boxoffice_gross_us,desc&start={var}&ref_=adv_nxt")
    soup = BS(page.content, 'html.parser')
    soup.prettify()
    info = soup.find_all(id = 'main')
    movie = info[0].find_all(class_='lister-item mode-advanced')
    movie_df = make_df(movie)
    insert_movies(movie_df)
    # #     print(movie_df)
    var +=50


In [31]:
movie_db = 'https://api.themoviedb.org/3/movie/550?api_key=0bf0d1d80e2ab08ed768c022e7f27155'

discover = '/discover/movie?primary_release_date.gte=2014-09-15&primary_release_date.lte=2014-10-22'

In [38]:
import requests

import json



In [36]:
response2 = requests.get().json()

response2

{'status_code': 7,
 'status_message': 'Invalid API key: You must be granted a valid key.',
 'success': False}

In [33]:
response = requests.get(movie_db).json()
response

ConnectionError: HTTPSConnectionPool(host='api.themoviedb', port=443): Max retries exceeded with url: /discover/movie?primary_release_date.gte=2014-09-15&primary_release_date.lte=2014-10-22 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x116f7e748>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [39]:
url = 'https://api.themoviedb.org/4/list/1'

api_key = '0bf0d1d80e2ab08ed768c022e7f27155'

headers = { f'Authorization': 'Bearer {api_key}'}

def api_call(url_params, api_key, request):
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()[request]
    return data

api_call() 


    
    
    
    
    
    
    

In [ ]:
redeict = https://www.themoviedb.org/redirect?external_source=imdb_id&external_id=tt2975590

In [40]:
get/find{external_id}

SyntaxError: invalid syntax (<ipython-input-40-0e57327959bc>, line 1)

In [ ]:
find_external_id = 'https://api.themoviedb.org/3/find/{external_id}?api_key=<<api_key>>&language=en-US&external_source=imdb_id'

In [3]:
cursor = db.cursor()

df = cursor.execute("""SELECT imdb_id FROM Movies.imdb_movies2;""")

df = pd.DataFrame(cursor.fetchall())

df.columns = [x[0] for x in cursor.description]

list_of_ids = df.values.tolist()

list_of_imdb_ids = [val[0] for val in list_of_ids]

list_of_imdb_ids

['tt0437086',
 'tt0448115',
 'tt0837563',
 'tt0859635',
 'tt10003978',
 'tt10011448',
 'tt10026172',
 'tt10044704',
 'tt10074426',
 'tt10078290',
 'tt10122042',
 'tt10125662',
 'tt10132048',
 'tt10140002',
 'tt10155342',
 'tt10156112',
 'tt10161238',
 'tt10208198',
 'tt10218674',
 'tt10234494',
 'tt1025100',
 'tt10257334',
 'tt10260672',
 'tt10279010',
 'tt10295602',
 'tt1034415',
 'tt10346206',
 'tt10383464',
 'tt10405390',
 'tt10407900',
 'tt10437138',
 'tt10442108',
 'tt10443252',
 'tt10461352',
 'tt10471420',
 'tt10521814',
 'tt10545076',
 'tt10550884',
 'tt10627720',
 'tt10637682',
 'tt10648440',
 'tt10648550',
 'tt10678786',
 'tt1072748',
 'tt10777384',
 'tt10889642',
 'tt10954574',
 'tt1137450',
 'tt1206885',
 'tt1213641',
 'tt1226837',
 'tt1235187',
 'tt1255919',
 'tt1259528',
 'tt1270797',
 'tt1273221',
 'tt1285009',
 'tt1298644',
 'tt1308728',
 'tt1318517',
 'tt1365519',
 'tt1389098',
 'tt1413492',
 'tt1477834',
 'tt1478839',
 'tt1489887',
 'tt1502407',
 'tt1517451',
 'tt1560

In [ ]:
# api_key = '0bf0d1d80e2ab08ed768c022e7f27155'
# all_info = []
# for id_ in list_of_imdb_ids:
#     find_external_id = requests.get(f'https://api.themoviedb.org/3/find/{id_}?api_key=0bf0d1d80e2ab08ed768c022e7f27155&language=en-US&external_source=imdb_id').json()
#     find_external_id['imdb_id']=id_
#     all_info.append(find_external_id)

# print(all_info[:5])

In [19]:
api_key = '0bf0d1d80e2ab08ed768c022e7f27155'
all_info = []
for id_ in list_of_imdb_ids:
    find_external_id = requests.get(f'https://api.themoviedb.org/3/find/{id_}?api_key=0bf0d1d80e2ab08ed768c022e7f27155&language=en-US&external_source=imdb_id').json()
    find_external_id['imdb_id']=id_
    all_info.append(find_external_id)
    print(all_info[:5])

[{'movie_results': [{'id': 399579, 'video': False, 'vote_count': 3685, 'vote_average': 6.9, 'title': 'Alita: Battle Angel', 'release_date': '2019-01-31', 'original_language': 'en', 'original_title': 'Alita: Battle Angel', 'genre_ids': [28, 12, 878, 53], 'backdrop_path': '/8yjqnpOfuFQg3qLRl9Ca1NgIBB5.jpg', 'adult': False, 'overview': 'When Alita awakens with no memory of who she is in a future world she does not recognize, she is taken in by Ido, a compassionate doctor who realizes that somewhere in this abandoned cyborg shell is the heart and soul of a young woman with an extraordinary past.', 'poster_path': '/xRWht48C2V8XNfzvPehyClOvDni.jpg', 'popularity': 34.006}], 'person_results': [], 'tv_results': [], 'tv_episode_results': [], 'tv_season_results': [], 'imdb_id': 'tt0437086'}]
[{'movie_results': [{'id': 399579, 'video': False, 'vote_count': 3685, 'vote_average': 6.9, 'title': 'Alita: Battle Angel', 'release_date': '2019-01-31', 'original_language': 'en', 'original_title': 'Alita: B

[{'movie_results': [{'id': 399579, 'video': False, 'vote_count': 3685, 'vote_average': 6.9, 'title': 'Alita: Battle Angel', 'release_date': '2019-01-31', 'original_language': 'en', 'original_title': 'Alita: Battle Angel', 'genre_ids': [28, 12, 878, 53], 'backdrop_path': '/8yjqnpOfuFQg3qLRl9Ca1NgIBB5.jpg', 'adult': False, 'overview': 'When Alita awakens with no memory of who she is in a future world she does not recognize, she is taken in by Ido, a compassionate doctor who realizes that somewhere in this abandoned cyborg shell is the heart and soul of a young woman with an extraordinary past.', 'poster_path': '/xRWht48C2V8XNfzvPehyClOvDni.jpg', 'popularity': 34.006}], 'person_results': [], 'tv_results': [], 'tv_episode_results': [], 'tv_season_results': [], 'imdb_id': 'tt0437086'}, {'movie_results': [{'id': 287947, 'video': False, 'vote_count': 3636, 'vote_average': 7.0, 'title': 'Shazam!', 'release_date': '2019-03-23', 'original_language': 'en', 'original_title': 'Shazam!', 'genre_ids'

[{'movie_results': [{'id': 399579, 'video': False, 'vote_count': 3685, 'vote_average': 6.9, 'title': 'Alita: Battle Angel', 'release_date': '2019-01-31', 'original_language': 'en', 'original_title': 'Alita: Battle Angel', 'genre_ids': [28, 12, 878, 53], 'backdrop_path': '/8yjqnpOfuFQg3qLRl9Ca1NgIBB5.jpg', 'adult': False, 'overview': 'When Alita awakens with no memory of who she is in a future world she does not recognize, she is taken in by Ido, a compassionate doctor who realizes that somewhere in this abandoned cyborg shell is the heart and soul of a young woman with an extraordinary past.', 'poster_path': '/xRWht48C2V8XNfzvPehyClOvDni.jpg', 'popularity': 34.006}], 'person_results': [], 'tv_results': [], 'tv_episode_results': [], 'tv_season_results': [], 'imdb_id': 'tt0437086'}, {'movie_results': [{'id': 287947, 'video': False, 'vote_count': 3636, 'vote_average': 7.0, 'title': 'Shazam!', 'release_date': '2019-03-23', 'original_language': 'en', 'original_title': 'Shazam!', 'genre_ids'

[{'movie_results': [{'id': 399579, 'video': False, 'vote_count': 3685, 'vote_average': 6.9, 'title': 'Alita: Battle Angel', 'release_date': '2019-01-31', 'original_language': 'en', 'original_title': 'Alita: Battle Angel', 'genre_ids': [28, 12, 878, 53], 'backdrop_path': '/8yjqnpOfuFQg3qLRl9Ca1NgIBB5.jpg', 'adult': False, 'overview': 'When Alita awakens with no memory of who she is in a future world she does not recognize, she is taken in by Ido, a compassionate doctor who realizes that somewhere in this abandoned cyborg shell is the heart and soul of a young woman with an extraordinary past.', 'poster_path': '/xRWht48C2V8XNfzvPehyClOvDni.jpg', 'popularity': 34.006}], 'person_results': [], 'tv_results': [], 'tv_episode_results': [], 'tv_season_results': [], 'imdb_id': 'tt0437086'}, {'movie_results': [{'id': 287947, 'video': False, 'vote_count': 3636, 'vote_average': 7.0, 'title': 'Shazam!', 'release_date': '2019-03-23', 'original_language': 'en', 'original_title': 'Shazam!', 'genre_ids'

[{'movie_results': [{'id': 399579, 'video': False, 'vote_count': 3685, 'vote_average': 6.9, 'title': 'Alita: Battle Angel', 'release_date': '2019-01-31', 'original_language': 'en', 'original_title': 'Alita: Battle Angel', 'genre_ids': [28, 12, 878, 53], 'backdrop_path': '/8yjqnpOfuFQg3qLRl9Ca1NgIBB5.jpg', 'adult': False, 'overview': 'When Alita awakens with no memory of who she is in a future world she does not recognize, she is taken in by Ido, a compassionate doctor who realizes that somewhere in this abandoned cyborg shell is the heart and soul of a young woman with an extraordinary past.', 'poster_path': '/xRWht48C2V8XNfzvPehyClOvDni.jpg', 'popularity': 34.006}], 'person_results': [], 'tv_results': [], 'tv_episode_results': [], 'tv_season_results': [], 'imdb_id': 'tt0437086'}, {'movie_results': [{'id': 287947, 'video': False, 'vote_count': 3636, 'vote_average': 7.0, 'title': 'Shazam!', 'release_date': '2019-03-23', 'original_language': 'en', 'original_title': 'Shazam!', 'genre_ids'

...

[{'movie_results': [{'id': 399579, 'video': False, 'vote_count': 3685, 'vote_average': 6.9, 'title': 'Alita: Battle Angel', 'release_date': '2019-01-31', 'original_language': 'en', 'original_title': 'Alita: Battle Angel', 'genre_ids': [28, 12, 878, 53], 'backdrop_path': '/8yjqnpOfuFQg3qLRl9Ca1NgIBB5.jpg', 'adult': False, 'overview': 'When Alita awakens with no memory of who she is in a future world she does not recognize, she is taken in by Ido, a compassionate doctor who realizes that somewhere in this abandoned cyborg shell is the heart and soul of a young woman with an extraordinary past.', 'poster_path': '/xRWht48C2V8XNfzvPehyClOvDni.jpg', 'popularity': 34.006}], 'person_results': [], 'tv_results': [], 'tv_episode_results': [], 'tv_season_results': [], 'imdb_id': 'tt0437086'}, {'movie_results': [{'id': 287947, 'video': False, 'vote_count': 3636, 'vote_average': 7.0, 'title': 'Shazam!', 'release_date': '2019-03-23', 'original_language': 'en', 'original_title': 'Shazam!', 'genre_ids'

[{'movie_results': [{'id': 399579, 'video': False, 'vote_count': 3685, 'vote_average': 6.9, 'title': 'Alita: Battle Angel', 'release_date': '2019-01-31', 'original_language': 'en', 'original_title': 'Alita: Battle Angel', 'genre_ids': [28, 12, 878, 53], 'backdrop_path': '/8yjqnpOfuFQg3qLRl9Ca1NgIBB5.jpg', 'adult': False, 'overview': 'When Alita awakens with no memory of who she is in a future world she does not recognize, she is taken in by Ido, a compassionate doctor who realizes that somewhere in this abandoned cyborg shell is the heart and soul of a young woman with an extraordinary past.', 'poster_path': '/xRWht48C2V8XNfzvPehyClOvDni.jpg', 'popularity': 34.006}], 'person_results': [], 'tv_results': [], 'tv_episode_results': [], 'tv_season_results': [], 'imdb_id': 'tt0437086'}, {'movie_results': [{'id': 287947, 'video': False, 'vote_count': 3636, 'vote_average': 7.0, 'title': 'Shazam!', 'release_date': '2019-03-23', 'original_language': 'en', 'original_title': 'Shazam!', 'genre_ids'

[{'movie_results': [{'id': 399579, 'video': False, 'vote_count': 3685, 'vote_average': 6.9, 'title': 'Alita: Battle Angel', 'release_date': '2019-01-31', 'original_language': 'en', 'original_title': 'Alita: Battle Angel', 'genre_ids': [28, 12, 878, 53], 'backdrop_path': '/8yjqnpOfuFQg3qLRl9Ca1NgIBB5.jpg', 'adult': False, 'overview': 'When Alita awakens with no memory of who she is in a future world she does not recognize, she is taken in by Ido, a compassionate doctor who realizes that somewhere in this abandoned cyborg shell is the heart and soul of a young woman with an extraordinary past.', 'poster_path': '/xRWht48C2V8XNfzvPehyClOvDni.jpg', 'popularity': 34.006}], 'person_results': [], 'tv_results': [], 'tv_episode_results': [], 'tv_season_results': [], 'imdb_id': 'tt0437086'}, {'movie_results': [{'id': 287947, 'video': False, 'vote_count': 3636, 'vote_average': 7.0, 'title': 'Shazam!', 'release_date': '2019-03-23', 'original_language': 'en', 'original_title': 'Shazam!', 'genre_ids'

[{'movie_results': [{'id': 399579, 'video': False, 'vote_count': 3685, 'vote_average': 6.9, 'title': 'Alita: Battle Angel', 'release_date': '2019-01-31', 'original_language': 'en', 'original_title': 'Alita: Battle Angel', 'genre_ids': [28, 12, 878, 53], 'backdrop_path': '/8yjqnpOfuFQg3qLRl9Ca1NgIBB5.jpg', 'adult': False, 'overview': 'When Alita awakens with no memory of who she is in a future world she does not recognize, she is taken in by Ido, a compassionate doctor who realizes that somewhere in this abandoned cyborg shell is the heart and soul of a young woman with an extraordinary past.', 'poster_path': '/xRWht48C2V8XNfzvPehyClOvDni.jpg', 'popularity': 34.006}], 'person_results': [], 'tv_results': [], 'tv_episode_results': [], 'tv_season_results': [], 'imdb_id': 'tt0437086'}, {'movie_results': [{'id': 287947, 'video': False, 'vote_count': 3636, 'vote_average': 7.0, 'title': 'Shazam!', 'release_date': '2019-03-23', 'original_language': 'en', 'original_title': 'Shazam!', 'genre_ids'

[{'movie_results': [{'id': 399579, 'video': False, 'vote_count': 3685, 'vote_average': 6.9, 'title': 'Alita: Battle Angel', 'release_date': '2019-01-31', 'original_language': 'en', 'original_title': 'Alita: Battle Angel', 'genre_ids': [28, 12, 878, 53], 'backdrop_path': '/8yjqnpOfuFQg3qLRl9Ca1NgIBB5.jpg', 'adult': False, 'overview': 'When Alita awakens with no memory of who she is in a future world she does not recognize, she is taken in by Ido, a compassionate doctor who realizes that somewhere in this abandoned cyborg shell is the heart and soul of a young woman with an extraordinary past.', 'poster_path': '/xRWht48C2V8XNfzvPehyClOvDni.jpg', 'popularity': 34.006}], 'person_results': [], 'tv_results': [], 'tv_episode_results': [], 'tv_season_results': [], 'imdb_id': 'tt0437086'}, {'movie_results': [{'id': 287947, 'video': False, 'vote_count': 3636, 'vote_average': 7.0, 'title': 'Shazam!', 'release_date': '2019-03-23', 'original_language': 'en', 'original_title': 'Shazam!', 'genre_ids'

In [35]:


def make_df(all_info):
    mymoviedatabase_dataframe2 = pd.DataFrame({
        "vote_count": vote_count,
        "title": title,
        "release_date": release_date,
        "popularity": popularity,
        "overview": overview,
        "vote_count": vote_count
    })
    #return new_dataframe

